In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../ipsim'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ipsim import *

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


In [ ]:
class ECSRTTest:
    Ea  = 72750     # activation energy J/gmol
    R   = 8.314     # gas constant J/gmol/K
    k0  = 7.2e10    # Arrhenius rate constant 1/min
    dHr = -5.0e4    # Enthalpy of reaction [J/mol]
    rho = 1000.0    # Density [g/L]
    Cp  = 0.239     # Heat capacity [J/g/K]
    UA  = 5.0e4     # Heat transfer [J/min/K]
    
    V   = 100.0     # Volume [L]
    q   = 100.0     # Flowrate [L/min]
    cAi = 1.0       # Inlet feed concentration [mol/L]
    Ti  = 350.0     # Inlet feed temperature [K]
    Tc  = 300.0     # Coolant temperature [K]

    cA0 = 0.5;      # Initial concentration [mol/L]
    T0  = 350.0;    # Initial temperature [K]
    
    def k(T):
        return ECSRTTest.k0*np.exp(-ECSRTTest.Ea/ECSRTTest.R/T)

In [ ]:
class ExothermicContinuousStirredTankReactor0(ProcessNode):
    def __init__(self, name, *, cA0 = ECSRTTest.cA0, T0 = ECSRTTest.T0, V = ECSRTTest.V
                , dHr = ECSRTTest.dHr, rho = ECSRTTest.rho, Cp=ECSRTTest.Cp,UA = ECSRTTest.UA):
        super().__init__(name)
        self._cA = cA0
        self._T = T0
        self._V = V
        self._dHr = dHr
        self._rho=rho
        self._Cp = Cp
        self._UA = UA
        self.create_input("q")
        self.create_input("cAi")
        self.create_input("Ti")
        self.create_input("Tc")
        self.create_output("cA")
        self.create_output("T")
    
    def evaluate(self):
        i = self.inputs()
        q   = i["q"]()
        cAi = i["cAi"]()
        Ti  = i["Ti"]()
        Tc  = i["Tc"]()
        cA  = self._cA
        T   = self._T
        
        kcA = ECSRTTest.k(T)*cA
        dt = self._model.dt()

        qV    = q/self._V
        dHpC  = -self._dHr/(self._rho*self._Cp)
        UAVpC = self._UA/(self._V*self._rho*self._Cp)
     
        _dcA  = (qV*(cAi-cA) - kcA)*dt
        _dT  = (qV*(Ti-T)+dHpC*kcA+UAVpC*(Tc-T))*dt
        cA = cA + _dcA
        T  = T  + _dT
        
        self._cA = cA
        self._T = T

        self.set_result("cA",cA)
        self.set_result("T",T)

In [ ]:
def prepare_model():    
    processModel = ProcessModel("test",dt=0.005)
    processModel.add_node(ProcessInputNode("InletFeed", {"q":100,"cAi":1,"Ti":350}))
    processModel.add_node(ProcessInputNode("Coolant", {"Tc":290}))
    processModel.add_node(ExothermicContinuousStirredTankReactor0("ECSTR"))
    processModel.bond_nodes("ECSTR","q","InletFeed","q")
    processModel.bond_nodes("ECSTR","cAi","InletFeed","cAi")
    processModel.bond_nodes("ECSTR","Ti","InletFeed","Ti")
    processModel.bond_nodes("ECSTR","Tc","Coolant","Tc")
    
    return processModel

In [ ]:
def prepare_data_coolant_temperature(process_model, points_count,*,skip_iteration=0, pivots = 1):
    actual_points = points_count - skip_iteration
    x = np.empty((actual_points,2))
    y = np.empty((actual_points,1),dtype=np.dtype('B'))

    if skip_iteration > 0:
        for _ in range(skip_iteration):
            state = process_model.next_state(("ECSTR",))

    pivot_leght = int(actual_points/(pivots+1))
   
    for _ in range(actual_points):
        state = process_model.next_state(("ECSTR",))
        cA = state['ECSTR']['cA']
        T  = state['ECSTR']['T']

        if (_ > 0) and (_ % pivot_leght) == 0:
            inode = process_model.nodes()["Coolant"]
            Tc = 304 if inode.value("Tc") == 290 else 290
            inode.change_value("Tc",Tc)
            print(f"{_} new Coolant Temperature {Tc}")

        x[_] = cA, T
        y[_] = 1 if ((cA > 0.8) and (T < 350)) else 0

    return x, y

In [ ]:
def test_ml_model_coolant_temperature(process_model, ml_model, points_count,*,skip_iteration=0, pivots = 2):
    actual_points = points_count - skip_iteration
    x = np.empty((actual_points,2))
    y = np.empty((actual_points,1),dtype=np.dtype('B'))

    if skip_iteration > 0:
        for _ in range(skip_iteration):
            state = process_model.next_state(("ECSTR",))

    pivot_leght = int(actual_points/(pivots+1))
   
    for _ in range(actual_points):
        state = process_model.next_state(("ECSTR",))
        cA = state['ECSTR']['cA']
        T  = state['ECSTR']['T']

        if (_ > 0) and (_ % pivot_leght) == 0:
            inode = process_model.nodes()["Coolant"]
            Tc = 304 if inode.value("Tc") == 290 else 290
            inode.change_value("Tc",Tc)
            print(f"{_} new Coolant Temperature {Tc}")

        presult = ml_model.predict([[cA, T]])[0]
        if presult == 0:
            inode = process_model.nodes()["Coolant"]
            inode.change_value("Tc",290)
            print(f"{_} fix Coolant Temperature")

        x[_] = cA, T
        y[_] = 1 if ((cA > 0.8) and (T < 350)) else 0

    return x, y

In [ ]:
def show_data(x, y):
    plt.figure(figsize=(12,3))
    plt.subplot(1, 3, 1)
    plt.ylim(0, 1)
    plt.plot(x[:,0])
    plt.title('Concentration [mol/L]')
    plt.subplot(1, 3, 2)
    plt.ylim(200, 600)
    plt.plot(x[:,1])
    plt.title('Temperature [K]')
    plt.subplot(1, 3, 3)
    plt.ylim(-1, 2)
    plt.plot(y)
    plt.title('target')
    plt.show()

In [ ]:
def train_logistic(x,y):
    model = LogisticRegression()
    model.fit(x, np.ravel(y))
    return model

In [ ]:
process_model = prepare_model()
x,y = prepare_data_coolant_temperature(process_model, 5500, skip_iteration = 500, pivots=2)
show_data(x,y)

In [ ]:
ml_model = train_logistic(x,y)

In [ ]:
process_model = prepare_model()
x,y = test_ml_model_coolant_temperature(process_model,ml_model, 7500, skip_iteration = 500, pivots=3)
show_data(x,y)